In [1]:
import numpy as np
import networkx as nx
import time

In [2]:
# Import 2QAN compiler passes
from bench_arch import BenchArch
from heuristic_mapper import HeuristicMapper
from qrouter_depth import QuRouter

In [3]:
from qiskit import transpile
from qiskit.transpiler.passes import DenseLayout, SabreLayout
from qiskit.transpiler import CouplingMap
from qiskit.converters import *

from tket_compiler import TketCompile

In [4]:
def fill_partial_mapping(partial_map, coupling_map):
    final_map = partial_map.copy()
    n_qbts = len(partial_map)
    device_qubits = []
    for edge in coupling_map:
        for q in edge:
            if q not in device_qubits:
                device_qubits.append(q)
    unused_dqs = [q for q in device_qubits if q not in partial_map.values()]
    for i in range(len(unused_dqs)):
        final_map[n_qbts+i]=unused_dqs[i]
    return final_map

## Run 2QAN compiler passes

In [5]:
def qs_compiler(benchmark, coupling_map, lattice_xy, trials=1, mapper='qap', bgate='cx', basis_gates=None, recompile=True):
    qs_circ = None
    qs_swap = [0, 0]
    qs_g2 = 0
    if recompile:
        for trial in range(trials):
            # Initial qubit mapping 
            start = time.time()
            if mapper == 'qap':
                # The default mapper based on Quadratic Assignment Problem
                mapper = HeuristicMapper(benchmark.copy(), lattice_xy=lattice_xy, coupling_map=coupling_map)
                init_map, cost = mapper.run(output='mapper', num_iter=200, lst_len=20)
            elif mapper == 'qiskit':
                # The mapper in Qiskit
                mapper = SabreLayout(CouplingMap(coupling_map),max_iterations=5)
                mapper.run(circuit_to_dag(benchmark.copy()))
                vpmap = mapper.property_set["layout"].get_virtual_bits()
                init_map0 = {}
                for k,v in vpmap.items():
                    init_map0[k.index]=v
                init_map = fill_partial_mapping(init_map0, coupling_map)
            elif mapper == 'tket':
                # The mapper in tket
                tket_mapper = TketCompile(benchmark.copy(), lattice_xy=lattice_xy, coupling_map=coupling_map)
                init_map = tket_mapper.placement()
            end = time.time()
            print("Mapper run time: ", end - start)
            # Routing and scheduling, takes init_map as input
            # init_map = {circuit qubit index:device qubit index}
            print('The inital qubit map is \n', init_map)
            router = QuRouter(benchmark.copy(), lattice_xy=lattice_xy, init_map=init_map, coupling_map=coupling_map)
            qs_circ0, swaps1 = router.run(layers=1, msmt='True')
            
            # Gate count 
            qs_circ1 = transpile(qs_circ0, basis_gates=None, optimization_level=3)
            g2_count1 = 0
            g2_count1 += qs_circ1.count_ops()[bgate]
            if 'unitary' in qs_circ1.count_ops():
                g2_count1 += qs_circ1.count_ops()['unitary']
            if 'swap' in qs_circ1.count_ops():
                g2_count1 += qs_circ1.count_ops()['swap']
            if trial == 0:
                qs_circ = qs_circ1
                qs_swap = swaps1
                qs_g2 = g2_count1 
            elif g2_count1 < qs_g2:
                qs_circ = qs_circ1
                qs_swap = swaps1
                qs_g2 = g2_count1
    else:
        qs_circ = transpile(benchmark.copy(), basis_gates=basis_gates, optimization_level=3)
        qs_g2 += qs_circ.count_ops()[bgate] 
        if 'unitary' in qs_circ.count_ops().keys():
            qs_g2 += qs_circ.count_ops()['unitary']
    return qs_circ, qs_swap, qs_g2

## Tests

In [6]:
import os
import pickle as pkl
import qiskit
# Benchmarks
# with open(os.path.join('qaoa_qasms.pkl'), 'rb') as f:
#     qasms = pkl.load(f)
with open(os.path.join('nnn_ising_qasms.pkl'), 'rb') as f:
    qasms = pkl.load(f)
# qasms = []
# for c in qaoas:
#     qasms.append(c.qasm())
circs = [qiskit.QuantumCircuit.from_qasm_str(qasm) for qasm in qasms]
test_circ = circs[-2]
# with open(os.path.join('nnn_ising_qasms.pkl'), 'wb') as f:
#     pkl.dump(qasms, f)  

In [7]:
# Device information
# gate set
basis_gates = ['id', 'rz', 'u3', 'u2', 'cx', 'reset']

# topology
qn = len(test_circ.qubits)
dx = int(np.sqrt(qn))
print('The number of qubits is ', qn)
if dx*dx >= qn:
    lattice_xy = (dx, dx)
elif dx*(dx+1) >= qn:
    lattice_xy = (dx, dx+1)
elif dx*(dx+2) >= qn:
    lattice_xy = (dx, dx+2)
grid_topology = BenchArch(test_circ, lattice_xy=lattice_xy).topology
coupling_map = [list(edge) for edge in list(grid_topology.edges)]
coupling_map += [[edge[1], edge[0]] for edge in list(grid_topology.edges)]

The number of qubits is  40


## QAP mapper + 2QAN routing&scheduling

In [8]:
qs_circ, qs_swap, qs_g2 = qs_compiler(test_circ, coupling_map, lattice_xy, bgate='rzz', basis_gates=basis_gates, recompile=True)
print('The number of SWAPs: ', qs_swap)

Mapper run time:  256.94903111457825
The inital qubit map is 
 {0: 8, 1: 20, 2: 14, 3: 21, 4: 15, 5: 16, 6: 17, 7: 11, 8: 5, 9: 10, 10: 4, 11: 9, 12: 3, 13: 2, 14: 1, 15: 7, 16: 6, 17: 13, 18: 12, 19: 18, 20: 24, 21: 30, 22: 36, 23: 31, 24: 37, 25: 25, 26: 38, 27: 26, 28: 32, 29: 27, 30: 33, 31: 39, 32: 40, 33: 41, 34: 34, 35: 35, 36: 28, 37: 29, 38: 22, 39: 23, 40: 0, 41: 19}
The number of SWAPs:  (22, 17)


In [9]:
qs_circ2, qs_swap, qs_g2 = qs_compiler(qs_circ, coupling_map, lattice_xy, bgate='cx', basis_gates=basis_gates, recompile=False)
print('The number of CNOTs: ', qs_g2)

The number of CNOTs:  186


## CQ tket mapper + 2QAN routing&scheduling

In [10]:
tket_circ, tket_swap, tket_g2 = qs_compiler(test_circ, coupling_map, lattice_xy, bgate='rzz', mapper='tket', basis_gates=basis_gates, recompile=True)
print('The number of SWAPs: ', tket_swap)

Mapper run time:  0.02393627166748047
The inital qubit map is 
 {0: 6, 1: 7, 2: 8, 3: 2, 4: 3, 5: 4, 6: 5, 7: 11, 8: 10, 9: 9, 10: 15, 11: 14, 12: 13, 13: 12, 14: 18, 15: 19, 16: 20, 17: 21, 18: 22, 19: 16, 20: 17, 21: 23, 22: 29, 23: 28, 24: 27, 25: 26, 26: 25, 27: 24, 28: 30, 29: 31, 30: 32, 31: 33, 32: 34, 33: 35, 34: 41, 35: 40, 36: 39, 37: 38, 38: 37, 39: 36, 40: 0, 41: 1}
The number of SWAPs:  (29, 16)


In [11]:
tket_circ2, tket_swap, tket_g2 = qs_compiler(tket_circ, coupling_map, lattice_xy, bgate='cx', basis_gates=basis_gates, recompile=False)
print('The number of CNOTs: ', tket_g2)

The number of CNOTs:  209


## Tket compiler

In [12]:
tket = TketCompile(test_circ, coupling_map=coupling_map)
tket_circ = tket.default_tket()
tket_cx = tket_circ.count_ops()['cx']
print('The number of CNOTs: ', tket_cx)

The number of CNOTs:  403


## Qiskit SABRE mapper (by Gushu Li) + 2QAN routing&scheduling

In [13]:
ibm_circ, ibm_swap, ibm_g2 = qs_compiler(test_circ, coupling_map, lattice_xy, trials=1, bgate='rzz', mapper='qiskit', basis_gates=basis_gates, recompile=True)
print('The number of SWAPs: ', ibm_swap)

<ipython-input-5-73957886cf9d>:20: DeprecationWarning: Back-references to from Bit instances to their containing Registers have been deprecated. Instead, inspect Registers to find their contained Bits.
  init_map0[k.index]=v


Mapper run time:  1.7593185901641846
The inital qubit map is 
 {0: 36, 1: 30, 2: 24, 3: 18, 4: 12, 5: 6, 6: 1, 7: 14, 8: 13, 9: 7, 10: 8, 11: 2, 12: 3, 13: 15, 14: 16, 15: 9, 16: 4, 17: 10, 18: 17, 19: 5, 20: 11, 21: 23, 22: 22, 23: 21, 24: 20, 25: 26, 26: 25, 27: 19, 28: 31, 29: 38, 30: 32, 31: 37, 32: 39, 33: 33, 34: 27, 35: 28, 36: 40, 37: 35, 38: 34, 39: 41, 40: 0, 41: 29}
The number of SWAPs:  (26, 16)


In [14]:
ibm_circ2, ibm_swap, ibm_g2 = qs_compiler(ibm_circ, coupling_map, lattice_xy, bgate='cx', basis_gates=basis_gates, recompile=False)
print('The number of CNOTs: ', ibm_g2)

The number of CNOTs:  200


In [15]:
from multi_decomposer import run_decompose
ibm_circ3, ibm_g3 = run_decompose(ibm_circ, bgate='iswap', params=[np.pi/2, 0], num_threads=1)
print('The number of iSWAPs: ', ibm_g3)

The number of iSWAPs:  200


In [16]:
ibm_circ4, ibm_g4 = run_decompose(ibm_circ, bgate='syc', params=[np.pi/2, np.pi/6], num_threads=1)
print('The number of SYCs: ', ibm_g4)

The number of SYCs:  200


## Qiskit compiler

In [17]:
qiskit_circ = transpile(test_circ, basis_gates=basis_gates, coupling_map=coupling_map, optimization_level=3)
print('The number of CNOTs: ', qiskit_circ.count_ops()['cx'])

The number of CNOTs:  624
